In [8]:
import akshare as ak
import json
import pandas as pd
import sys
import datetime
import os
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode
from pyecharts import options as opts
from pyecharts.charts import Scatter
from pyecharts.options import InitOpts
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line
import matplotlib.pyplot as plt
from pyecharts import options as opts
from pyecharts.charts import Bar
%matplotlib inline


def convert_time(x):
    return str(x).replace(' 00:00:00','')
    
import re
sys.path.append('..')
from configure.settings import DBSelector
db = DBSelector()
mongo = db.mongo('qq')
fund = mongo['fund']

In [3]:
def get_net_value(code):
    fund_open_fund_info_em_df = ak.fund_open_fund_info_em(fund=code, indicator="累计净值走势")
    return fund_open_fund_info_em_df

In [7]:


def dump_mongo(df,code):
    js_str = df.to_json(orient='index')
    js_dict = json.loads(js_str)
    js_list = js_dict.values()
    try:
        fund[code].insert_many(js_list)
    except Exception as e:
        print(e)
        


In [9]:
def start_crawl(code):
    df = get_net_value(code)
    df['净值日期']=df['净值日期'].astype(str)
    dump_mongo(df,code)


In [10]:
txt='''
1. $西部利得国企红利指数增强A(F501059)$ 

跟踪中证国有企业红利指数，基金经理@盛丰衍 等，今年以来回报16.40%，近一年回报5.47%，近一年最大回撤-19.28%，二季度末规模2.98亿。

2. $创金合信中证红利低波动指数A(F005561)$ 

跟踪中证红利低波动指数，今年以来回报10.94%，近一年回报5.81%，近一年最大回撤-15.67%，二季度末规模3.81亿。

3. $万家中证红利指数(LOF)A(F161907)$ 

跟踪中证红利指数，今年以来回报4.14%，近一年回报-3.07%，近一年最大回撤-17.41%，二季度末规模1.32亿。

4. $华宝红利基金(F501029)$ 

跟踪中信标普中国A股红利机会指数，今年以来回报1.32%，近一年回报3.95%，近一年最大回撤-12.60%，二季度末规模8.79亿。

5.$泰达消费红利指数A(F008928)$ 

跟踪中证主要消费红利指数，今年以来回报-5.82%，近一年回报3.20%，近一年最大回撤-18.13%，二季度末规模3.34亿。

6. $中欧红利优享混合A(F004814)$ 

基金经理，今年以来回报-6.54%，近一年回报-4.85%，近一年最大回撤-22.23%，二季度末规模17.06亿。

7. $融通红利机会混合A(F005618)$ 

基金经理何龙，今年以来回报-4.82%，近一年回报-4.22%，近一年最大回撤-9.08%，二季度末规模0.16亿。

8. $长盛量化红利混合(F080005)$ 

基金经理王宁，今年以来回报-0.36%，近一年回报-8.66%，近一年最大回撤-17.97%，二季度末规模2.07亿。

9. $南方大盘红利低波50ETF联接A(F008163)$ 

跟踪标普中国A股大盘红利低波50指数，今年以来回报-3.54%，近一年回报-3.57%，近一年最大回撤-12.75%，二季度末规模0.45亿。

10. $财通中证香港红利等权指数A(F006658)$ 

跟踪中证香港红利等权投资指数，今年以来回报-8.09%，近一年回报-20.85%，近一年最大回撤-22.53%，二季度末规模0.16亿。

11. $景顺长城中证沪港深红利低波A(F007751)$ 

跟踪中证沪港深红利成长低波动人民币指数，今年以来回报1.71%，近一年回报-0.97%，近一年最大回撤-12.92%，二季度末规模0.80亿

'''

code_list = re.findall('\(F(\d+)\)',txt)
fund_name = re.findall('\$(.*?)\$',txt)
fund_dict = dict(zip(code_list,fund_name))

In [6]:
fund_dict

{'501059': '西部利得国企红利指数增强A(F501059)',
 '005561': '创金合信中证红利低波动指数A(F005561)',
 '161907': '万家中证红利指数(LOF)A(F161907)',
 '501029': '华宝红利基金(F501029)',
 '008928': '泰达消费红利指数A(F008928)',
 '004814': '中欧红利优享混合A(F004814)',
 '005618': '融通红利机会混合A(F005618)',
 '080005': '长盛量化红利混合(F080005)',
 '008163': '南方大盘红利低波50ETF联接A(F008163)',
 '006658': '财通中证香港红利等权指数A(F006658)',
 '007751': '景顺长城中证沪港深红利低波A(F007751)'}

In [11]:
for code,name in fund_dict.items():
    start_crawl(code)

In [60]:
# after fetch data

In [13]:
def plot_profit_line(df,code):
    title="{}基金收益率曲线".format(code)
    X=df['净值日期'].tolist()
    Y=list(map(lambda x:round(x,2),df['单位净值'].tolist()))
    c = (
        Line()
        .add_xaxis(X)
        .add_yaxis('', Y, is_smooth=True,
            label_opts=opts.LabelOpts(is_show=False),
            linestyle_opts=opts.LineStyleOpts(width=1,color='rgb(255, 0, 0)'),
        ).set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(
                                    name='日期',
                                    # min_interval=5,
                                    splitline_opts=opts.SplitLineOpts(is_show=True),
                                            axislabel_opts=opts.LabelOpts(rotate=45),

                                    ),
            yaxis_opts=opts.AxisOpts(
                                    min_=min(Y),
                                    max_=max(Y),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            )
                                        ).set_colors(['green'])
        .render(f"../plot_image/{title}.html")
    )

In [14]:

for code,name in fund_dict.items():
    result = []
    for item in fund[code].find({},{'_id':0}):
        result.append(item)
    df = pd.DataFrame(result)
    plot_profit_line(df,name)
    

In [15]:
name_list  = list(fund_dict.values())

In [16]:
sipmle_name = [i.split('(')[0] for i in name_list]

In [17]:
sipmle_name

['西部利得国企红利指数增强A',
 '创金合信中证红利低波动指数A',
 '万家中证红利指数',
 '华宝红利基金',
 '泰达消费红利指数A',
 '中欧红利优享混合A',
 '融通红利机会混合A',
 '长盛量化红利混合',
 '南方大盘红利低波50ETF联接A',
 '财通中证香港红利等权指数A',
 '景顺长城中证沪港深红利低波A']

In [31]:
W=10000
focus_num = [
2.7*W,
1.7*W,
1.2*W,
1.2*W,
2.9*W,
11.2*W,
9.3*W,
1569,
2191,
]

In [56]:
c = (
    Scatter(InitOpts(width='800px',height='500px'))
    .add_xaxis(sipmle_name)
    .add_yaxis('A',focus_num,label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
                                    # name='日期',
                                    # min_interval=5,
                                    splitline_opts=opts.SplitLineOpts(is_show=True),
                                            axislabel_opts=opts.LabelOpts(rotate=-45),

                                    ),
        yaxis_opts=opts.AxisOpts(    
            min_=0,
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
        title_opts=opts.TitleOpts(title="welcome"),
        visualmap_opts=opts.VisualMapOpts(type_="size",max_=150000, min_=2000),
    )
    .render("welcome_fund.html")
)

In [47]:
focus_num

[27000.0, 17000.0, 12000.0, 12000.0, 29000.0, 112000.0, 93000.0, 1569, 2191]

In [96]:
grid = (
    Grid(init_opts=opts.InitOpts())
    .add(c, grid_opts=opts.GridOpts(pos_bottom="40%"))
    .render("grid_vertical.html")
)

In [95]:

c = (
    Bar()
    .add_xaxis(
    new_name
    )
    .add_yaxis("基金关注度", focus_num,label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0)),
        title_opts=opts.TitleOpts(title="基金关注度"),
    )
    # .render("基金关注度.html")
)

In [72]:
new_name = []
for name in sipmle_name:
    new_name.append('\n'.join(list(name)))

In [73]:
new_name

['华\n安\n纳\n斯\n达\n克\n1\n0\n0\n指\n数',
 '博\n时\n标\n普\n5\n0\n0\nE\nT\nF\n联\n接',
 '天\n弘\n恒\n生\n科\n技\n指\n数\nA',
 '嘉\n实\n港\n股\n通\n新\n经\n济\n指\n数\nA',
 '华\n夏\n恒\n生\nE\nT\nF\n联\n接',
 '易\n方\n达\n中\n概\n互\n联\n5\n0\nE\nT\nF\n联\n接\n人\n民\n币\nA',
 '交\n银\n中\n证\n海\n外\n中\n国\n互\n联\n网\n指\n数',
 '华\n安\n香\n港\n精\n选',
 '中\n欧\n丰\n泓\n沪\n港\n深\nA']

In [71]:
sipmle_name[0]


'华安纳斯达克100指数'

In [115]:
date='20190118'
year = (datetime.datetime.now() - datetime.datetime.strptime(date,'%Y%m%d')).days/365
print(year)

3.0904109589041098


In [116]:
0.98**(1/3.09)

0.9934832338426756

In [100]:
for i in sipmle_name:
    print(i)

华安纳斯达克100指数
博时标普500ETF联接
天弘恒生科技指数A
嘉实港股通新经济指数A
华夏恒生ETF联接
易方达中概互联50ETF联接人民币A
交银中证海外中国互联网指数
华安香港精选
中欧丰泓沪港深A


In [111]:
1-0.7228

0.2772

In [114]:
0.5923-1

-0.40769999999999995

In [117]:
0.9934832338426756-1

-0.006516766157324438

In [3]:
code='006327'
result=[]
for item in fund[code].find({},{'_id':0}):
    result.append(item)
df = pd.DataFrame(result)

In [4]:
df.head()

,净值日期,单位净值,日增长率
0,2019-01-18,1.0000,0.0
1,2019-01-25,1.0001,0.0
2,2019-02-01,1.0003,0.0
3,2019-02-15,1.0007,0.0
4,2019-02-18,1.0007,0.0


['2019', '01', '18']

In [10]:
def get_net_value_df(code):
    result=[]
    for item in fund[code].find({},{'_id':0}):
        result.append(item)
    df = pd.DataFrame(result)
    return df

In [30]:
def automatic_investment_plan(code):
    df = get_net_value_df(code)

    money = 10000
    total_share =0 
    interval = 22
    length = len(df)
    sum_money=0
    count=0
    for i in range(0,length,interval):
        buy_date_df = df.iloc[i]
        share = money/buy_date_df['单位净值']
        total_share+=share
        sum_money+=money
        count+=1
    virtual_profit = (df.iloc[-1]['单位净值']*total_share-sum_money)/sum_money*100
    data=[]
    year,month,day=df.iloc[0]['净值日期'].split('-')
    for i in range(count):
        data.append((datetime.date(int(year), int(month), int(day))+datetime.timedelta(days=i*30), -1*money))
    current_money = df.iloc[-1]['单位净值']*total_share
    data.append((datetime.date(int(year), int(month), int(day))+datetime.timedelta(days=i*30),current_money))
    percent = xirr(data)
    print(sum_money,round(current_money,2))
    every_round_profit = irr([money]*count+[-1*current_money])
    real_profit = pow(every_round_profit+1,count)-1
    print(code,round(percent*100,2),round(virtual_profit,2),round(real_profit*100,2))

In [2]:
from datetime import date
 
# 函数
def xirr(cashflows):
    years = [(ta[0] - cashflows[0][0]).days / 365. for ta in cashflows]
    residual = 1.0
    step = 0.05
    guess = 0.05
    epsilon = 0.0001
    limit = 10000
    while abs(residual) > epsilon and limit > 0:
        limit -= 1
        residual = 0.0
        for i, trans in enumerate(cashflows):
            residual += trans[1] / pow(guess, years[i])
        if abs(residual) > epsilon:
            if residual > 0:
                guess += step
            else:
                guess -= step
                step /= 2.0
    return guess - 1


In [ ]:
# 测试
data = [(datetime.date(2006, 1, 24), -39967), (datetime.date(2008, 2, 6), -19866), (datetime.date(2010, 10, 18), 245706), (datetime.date(2013, 9, 14), 52142)]
xirr(data)

In [31]:
for code in code_list:
    automatic_investment_plan(code)

950000 2140678.11
040046 20.63 125.33 322.0
1040000 1903879.5
050025 13.96 83.07 200.15
70000 60075.1
012348 -47.41 -14.18 -23.88
340000 306207.33
501311 -7.57 -9.94 -18.69
1020000 1054546.22
000071 0.8 3.39 6.78
340000 280241.33
006327 -13.74 -17.58 -32.14
740000 665284.69
164906 -3.55 -10.1 -19.25
1260000 2254157.95
040018 10.95 78.9 189.16
580000 747497.07
002685 10.95 28.88 61.6


In [16]:
import numpy as np

In [19]:
def irr(values):
    """
    Return the Internal Rate of Return (IRR).
    .. deprecated:: 1.18
       `irr` is deprecated; for details, see NEP 32 [1]_.
       Use the corresponding function in the numpy-financial library,
       https://pypi.org/project/numpy-financial.
    This is the "average" periodically compounded rate of return
    that gives a net present value of 0.0; for a more complete explanation,
    see Notes below.
    :class:`decimal.Decimal` type is not supported.
    Parameters
    ----------
    values : array_like, shape(N,)
        Input cash flows per time period.  By convention, net "deposits"
        are negative and net "withdrawals" are positive.  Thus, for
        example, at least the first element of `values`, which represents
        the initial investment, will typically be negative.
    Returns
    -------
    out : float
        Internal Rate of Return for periodic input values.
    Notes
    -----
    The IRR is perhaps best understood through an example (illustrated
    using np.irr in the Examples section below).  Suppose one invests 100
    units and then makes the following withdrawals at regular (fixed)
    intervals: 39, 59, 55, 20.  Assuming the ending value is 0, one's 100
    unit investment yields 173 units; however, due to the combination of
    compounding and the periodic withdrawals, the "average" rate of return
    is neither simply 0.73/4 nor (1.73)^0.25-1.  Rather, it is the solution
    (for :math:`r`) of the equation:
    .. math:: -100 + \\frac{39}{1+r} + \\frac{59}{(1+r)^2}
     + \\frac{55}{(1+r)^3} + \\frac{20}{(1+r)^4} = 0
    In general, for `values` :math:`= [v_0, v_1, ... v_M]`,
    irr is the solution of the equation: [2]_
    .. math:: \\sum_{t=0}^M{\\frac{v_t}{(1+irr)^{t}}} = 0
    References
    ----------
    .. [1] NumPy Enhancement Proposal (NEP) 32,
       https://numpy.org/neps/nep-0032-remove-financial-functions.html
    .. [2] L. J. Gitman, "Principles of Managerial Finance, Brief," 3rd ed.,
       Addison-Wesley, 2003, pg. 348.
    Examples
    --------
    >>> round(np.irr([-100, 39, 59, 55, 20]), 5)
    0.28095
    >>> round(np.irr([-100, 0, 0, 74]), 5)
    -0.0955
    >>> round(np.irr([-100, 100, 0, -7]), 5)
    -0.0833
    >>> round(np.irr([-100, 100, 0, 7]), 5)
    0.06206
    >>> round(np.irr([-5, 10.5, 1, -8, 1]), 5)
    0.0886
    """
    # `np.roots` call is why this function does not support Decimal type.
    #
    # Ultimately Decimal support needs to be added to np.roots, which has
    # greater implications on the entire linear algebra module and how it does
    # eigenvalue computations.
    res = np.roots(values[::-1])  # 求根，对于n次多项式，p[0] * x**n + p[1] * x**(n-1) + ... + p[n-1]*x + p[n]，传入p的列表参数[p[0],p[1],...p[n]].
    mask = (res.imag == 0) & (res.real > 0)  # 虚部为0，实部为非负数。
    if not mask.any():  # 判断是否有满足条件的实根
        return np.nan  # 不满足，返回Not A Number
    res = res[mask].real
    # NPV(rate) = 0 can have more than one solution so we return
    # only the solution closest to zero.
    rate = 1/res - 1  # 这里解出的res，也就是符合条件的x，其实等于1/(1+r)，因此要做一个变换回去，r=1/x-1
    rate = rate.item(np.argmin(np.abs(rate)))  # argmin()取最小值的下标，也就是说可能会计算出多个折现率，我们取最小那个
    return rate